## <center>Tensorflow 实战 RNN</center>

### 1 单步RNN: RNNCell

RNNCell 它是 Tensorflow 中实现 RNN 的基本单元,每个 RNNCell 都有一个call 方法,使用方式:
```python
(output,next_state) = call(input,state)
```
借助图片来说可能更容易理解。假设我们有一个初始状态 $h_0$，还有输入 $x_1$，调用 call($x_1$, $h_0$)后就可以得到($output_1$, $h_1$)：
<br/><br/>
<center><img src="./img/9/1.jpg" width="600"/></center>
<br/><br/>
再调用一次 call($x_1$,$h_1$) 就可以得到 ($output_2,h_2$)
<br/><br/>
<center><img src="./img/9/2.jpg" width="600"/></center>

也就是说，每调用一次 RNNCell 的 call 方法，就相当于在时间上“推进了一步”，这就是 RNNCell 的基本功能。除了call方法外，对于RNNCell，还有两个类属性比较重要：
+ state_size
+ output_size

前者是隐层的大小，后者是输出的大小。比如我们通常是将一个 batch 送入模型计算，设输入数据的形状为(batch_size, input_size)，那么计算时得到的隐层状态就是(batch_size, state_size)，输出就是(batch_size, output_size)。

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
gpu_no = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_no
# 定义TensorFlow配置
config = tf.ConfigProto()
# 配置GPU内存分配方式，按需增长，很关键
config.gpu_options.allow_growth = True
# 配置可使用的显存比例
config.gpu_options.per_process_gpu_memory_fraction = 0.3
# 在创建session的时候把config作为参数传进去
sess = tf.Session(config = config)

In [2]:
cell = tf.contrib.rnn.BasicRNNCell(num_units=128) # state_size = 128
print(cell.state_size) # 128

inputs = tf.placeholder(np.float32, shape=(32, 100)) # 32 是 batch_size
h0 = cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态，形状为(batch_size, state_size)
output1, h1 = cell.__call__(inputs, h0) #调用call函数

print(h1.shape) # (32, 128)

128
(32, 128)


对于BasicLSTMCell，情况有些许不同，<font color='red'>因为LSTM可以看做有两个隐状态h和c</font>，对应的隐层就是一个Tuple，每个都是(batch_size, state_size)的形状：

In [3]:
lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=128)
inputs = tf.placeholder(np.float32, shape=(32, 100)) # 32 是 batch_size
h0 = lstm_cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态
output, h1 = lstm_cell.__call__(inputs, h0)

print(h1.h)  # shape=(32, 128)
print(h1.c)  # shape=(32, 128)

Tensor("basic_lstm_cell/Mul_2:0", shape=(32, 128), dtype=float32)
Tensor("basic_lstm_cell/Add_1:0", shape=(32, 128), dtype=float32)


### 2 多步RNN: tf.nn.dynamic_rnn

基础的 RNNCell 有一个很明显的问题：对于单个的 RNNCell，我们使用它的 call 函数进行运算时，只是在序列时间上前进了一步。比如使用 $x_1$、$h_0$ 得到 $h_1$，通过 $x_2$、$h_1$ 得到 $h_2$ 等。如果我们的序列长度为 10，就要调用 10 次 call 函数，比较麻烦。对此，TensorFlow 提供了一个 $tf.nn.dynamic\_rnn$ 函数，使用该函数就相当于调用了 $n$ 次 call 函数。即通过 {$h_0,x_1, x_2, …., x_n$} 直接得 {$h_1,h_2…,h_n$}。

具体来说，设我们输入数据的格式为 (batch_size, time_steps, input_size) ，其中 time_steps 表示序列本身的长度，如在 Char RNN 中，长度为 10 的句子对应的 time_steps 就等于10。最后的 input_size 就表示输入数据单个序列单个时间维度上固有的长度。另外我们已经定义好了一个 RNNCell，调用该 RNNCell 的 call 函数 time_steps 次

In [4]:
tf.reset_default_graph()
batch_size = 4 
# 通常输入数据的格式为 [timesteps,batch_size,input_data],其中input_data 表示当前RNN的输入. 
input = tf.random_normal(shape=[3, batch_size, 6], dtype=tf.float32)
# 这里使用最基本的 RNNCell ,10 表示一个RNNCell中有多少个隐藏单元
cell = tf.nn.rnn_cell.BasicRNNCell(10)
# 第一个RNNCell的输入为,输入数据和一个初始化的隐含状态,这个隐含状态通常初始化为0
#　初始化状态的大小就是 batch_size, 表明有 batch_size 个独立的状态
init_state = cell.zero_state(batch_size, dtype=tf.float32)
# output 表示每一个RNNCell 的输出,输出的形状为(3,4,10)
# final_state 表示最后一个 RNNCell 的输出状态,输出形状为(4,10)
output, final_state = tf.nn.dynamic_rnn(cell, input, initial_state=init_state, time_major=True)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(output).shape)
    print(sess.run(final_state))
    print(sess.run(final_state).shape)

(3, 4, 10)
[[-0.84242034  0.14048211  0.940332    0.78976446 -0.03859949  0.72312653
   0.17395307  0.3693972  -0.55532706 -0.55590576]
 [ 0.6636888  -0.9847835  -0.85880333 -0.0067372  -0.46843302 -0.5927951
   0.07098266 -0.7169884   0.36924684  0.87033564]
 [ 0.11519817  0.71050143 -0.47727397  0.76597834 -0.26745605 -0.19413835
  -0.3841084  -0.36119905 -0.47373918 -0.98366785]
 [-0.90471053 -0.49273717  0.5766394  -0.89226353 -0.40110266  0.19063497
   0.2493871   0.02936358  0.86817366  0.6109415 ]]
(4, 10)


### 3 堆叠RNNCell: MultiRNNCell

很多时候，单层 RNN 的能力有限，我们需要多层的 RNN。将 $x$ 输入第一层 RNN 的后得到隐层状态 $h$,这个隐层状态就相当于第二层 RNN 的输入，第二层 RNN 的隐层状态又相当于第三层 RNN 的输入，以此类推。在TensorFlow 中，可以使用 $tf.nn.rnn\_cell.MultiRNNCell$ 函数对 RNNCell 进行堆叠，相应的示例程序如下：

In [5]:
tf.reset_default_graph()
# 每调用一次这个函数就返回一个BasicRNNCell
def get_a_cell():
    return tf.nn.rnn_cell.BasicRNNCell(num_units=128)
# 用tf.nn.rnn_cell MultiRNNCell创建3层RNN
cell = tf.nn.rnn_cell.MultiRNNCell([get_a_cell() for _ in range(3)]) # 3层RNN
# 得到的cell实际也是RNNCell的子类
# 它的state_size是(128, 128, 128)
# (128, 128, 128)并不是128x128x128的意思
# 而是表示共有3个隐层状态，每个隐层状态的大小为128
print(cell.state_size) # (128, 128, 128)
# 使用对应的call函数
inputs = tf.placeholder(np.float32, shape=(32, 100)) # 32 是 batch_size
h0 = cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态
output, h1 = cell.call(inputs, h0)
print(h1) # tuple中含有3个32x128的向量

(128, 128, 128)
(<tf.Tensor 'cell_0/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>, <tf.Tensor 'cell_1/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>, <tf.Tensor 'cell_2/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>)


### 4 Tensorflow 中的 cell 类

Tensorflow 中定义了 5 个常用的 cell 类,都可以在  $tf.contrib.rnn.XXXCell$  中找到, 具体定义如下:
+ BasicRNNCell:
```python
BasicRNNCell(num_units)
num_units: 每个 cell 包含的隐含单元个数
```
+ BasicLSTMCell:
```python
BasicLSTMCell(num_units,forget_bias=1.0,activation=tanh)
num_units: 每个 cell 包含的隐含单元个数
forget_bias: 添加到 forget 门的偏置
activation: 默认激活函数
```
+ LSTMCell:
```python
LSTMCell(num_units,use_peepholes=False,cell_cip=None,initializer=None,forget_bias=1.0)
num_units: 每个 cell 包含的隐含单元个数
cell_clip: 在输出前对 cell 状态按照给定值进行截断处理
initializer: 指定初始化函数
forget_bias: 添加到 forget 门的偏置
```
+ GRUCell:
```python
GRUCell(num_units)
num_units: 每个 cell 包含的隐含单元个数
```
+ MultiRNNCell:
```python
MultiRNNCell(cells,state_is_tuple=True)
cells: 一个 cell 列表,将列表中的 cell 一个个堆叠起来,如果使用 cells=[cell1,cell2],就是一共两层,数据经过 cell1 后还要经过 cell2
state_is_tuple: 如果是 True 则返回 n-tuple,将cell的输出值与状态组成一个 tuple .其中,输出值的结构为:c=[batch_size,num_units],输出状态的结构为 h=[batch_size,num_units]
```
<font color='red'>注意</font><br/>
在使用 MultiRNNCell 时,有些习惯的写法是 cells 参数直接使用 [cell]$\times$n 来代表创建 n 层的 cell, 这种写法如果不使用作用域隔离,则会报编译错误,或者使用一个外层循环将 cell 一个个 append 进去来解决命名冲突.

### 5 通过 cell 类构建 RNN

定义好 cell 类之后,还需要将他们链接起来构成 RNN 网络. Tensorflow 中主要有以下积累构建网络的模式,构建函数主要在两个包中,$tf.nn$ 和 $tf.contrib$ ,其中 $tf.contrib$ 表示由其他人提供的,而 $tf.nn$ 表示官方提供的稳定版:

#### (1) 静态 RNN 构建
Tensorflow 中提供了一个构建静态 RNN 的函数 $tf.nn.static\_rnn$, 定义如下:
```python
tf.nn.static_rnn(cell,inputs,initial_state=None,dtype=None,sequence_length=None,scope=None)
cell: 由 cell 类生成的对象
inputs: 输入数据,一般数据输入格式为(timesteps,batch_size,cell_data)
initial_state: 初始化 cell 状态
dtype: 期望输出和初始化 state 的类型
sequence_length: 每一个输入的序列长度,如果超过了 timesteps 就会截断,这也是静态 RNN 的一个缺点
返回值: 一个是结果,一个是 cell 状态,结果是一个list,输入是多少个时序,list 里面就会有多少个元素.
```

#### (2) 动态 RNN 构建
Tensorflow 中提供了一个构建动态 RNN 的函数 $tf.nn.dynamic\_rnn$, 定义如下:
```python
tf.nn.dynamic_rnn(cell,inputs,initial_state=None,dtype=None,sequence_length=None,scope=None)
cell: 由 cell 类生成的对象
inputs: 输入数据,一般数据输入格式为(batch_size,timesteps,cell_data)
initial_state: 初始化 cell 状态
dtype: 期望输出和初始化 state 的类型
sequence_length: 每一个输入的序列长度,如果超过了 timesteps 就会截断,这也是静态 RNN 的一个缺点
time_major: 默认值 False 时, input 的形状为 [batch_sise,timesteps,cell_data]. 如果是 True,形状为[timesteps,batch_size,cell_data]
返回值: 一个是结果,一个是 cell 状态,结果是以 [batch_size,timesteps,...] 形式的张量.
```
<font color="red">注意</font><br/>
动态 RNN 的输出,它是以 batch_size 优先的矩阵,因为我们需要取最后一个时序的输出,所以需要处理成一个以时间优先的形式.

#### (3) 双向 RNN 构建
双向 RNN 作为一个可以学习正,反向归来的循环神经网络,在 Tensorflow 中有3个函数可以使用:
+ tf.nn.bidirectional_dynamic_rnn
```python
bidirectional_dynamic_rnn(cell_fw,cell_bw,inputs,initial_states_fw,initial_states_bw)
cell_fw,cell_bw: 前向和反向的 Cell
inputs: 输入序列,一个张量输入,形状为[batch_size,timesteps,cell_data]
initial_states_fw,initial_states_bw: 前向和反向 cell 的初始状态
#
返回值: 是一个tuple(outputs,output_states),其中outputs也是一个
tuple(output_fw,output_bw),每个值是一个张量 [batch_size,timesteps,layers_output],如果需要总的结果,可以将前后项的 layers_output 使用tf.concat 连接起来
```

+ tf.contrib.rnn.stack_bidirectional_rnn
```python
创建一个静态的多层双向网络.输出作为下一层的输入,前后向的输入大小必须一致,两层之间是独立的,不能共享信息.
stack_bidirectional_rnn(cells_fw,cells_bw,inputs,initial_states_fw,initial_states_bw)
cells_fw,cells_bw: 前向和后向实例化之后的 cell 列表,正反向的 list 长度必须相等,输入必须相同
inputs: 输入为 [timesteps,batch_size,input_size] 形状的张量
initial_states_fw,initial_states_bw: 前向和后向的 cell 初始化状态
#
返回值: 是一个tuple(outputs,output_states),其中outputs为 
[timesteps,batch_size,layers_output] 形状的张量, layers_output 包含 tf.concat 之后的正反向的输出
```

+ tf.contrib.rnn.stack_bidirectional_dynamic_rnn
```python
创建一个动态的多层双向网络.输出作为下一层的输入,前后向的输入大小必须一致,两层之间是独立的,不能共享信息.
stack_bidirectional_dynamic_rnn(cells_fw,cells_bw,inputs,initial_states_fw,initial_states_bw)
cells_fw,cells_bw: 前向和后向实例化之后的 cell 列表,正反向的 list 长度必须相等,输入必须相同
inputs: 输入形状为 [batch_size,timesteps,input_size] 形状的张量,与静态的关键不同
initial_states_fw,initial_states_bw: 前向和后向的 cell 初始化状态
#
返回值: 是一个tuple(outputs,output_states),其中outputs为 
[batch_size,timesteps,layers_output] 形状的张量, layers_output 包含 tf.concat 之后的正反向的输出
```

<font color='red'>注意</font><br/>
在单层,双层,双向 RNN 函数的介绍中,都有动态和静态之分.静态的意思就是按照样本的时间序列个数 $n$ 展开,在图中创建 $n$ 个序列的 cell. 动态的意思是只创建样本中一个序列的 RNN,其他的序列数据都会通过循环来进入该 RNN 来运算.

通过静态生成的 RNN 网络,生成过程所需的时间比较长,网络所占有的内存会更多,导出的模型会更大.模型中会带有每个序列中间的状态信息,便于调试.并且所有样本的序列长度固定,过短需要填充,过长就会截断. 通过动态生成的 RNN 网络,所占的内存比较小,导出的模型较小.模型中只会有最后的状态,在使用过程中,支持不同的序列个数.

#### (4) 使用动态 RNN 处理变长序列
动态 RNN 还有个更高级的功能,就是可以处理变长序列,方法就是:<font color='red'>在准备样本的同时,将样本对应的长度也作为初始化参数,一起创建动态 RNN</font>

In [6]:
tf.reset_default_graph()
x = np.random.randn(2,4,5)
x = x.astype(np.float32)
x[1,1:]=0
print(x)

[[[-1.2893718  -1.3948148   0.58520323  0.97595936  1.9312968 ]
  [ 1.5639083   0.6806277  -0.39227822 -1.2727726  -0.6733243 ]
  [-1.7387964  -2.4447942  -0.61489654 -0.0258193   0.04097625]
  [ 0.2515934   1.9033813  -0.61183923  0.9252508  -0.76407474]]

 [[ 0.5319435   0.621407    0.14989032  0.45473146  1.1019354 ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]]]


In [7]:
seq_lengths = [4,1]
# 分别建立一个 lstm 和 gru 的 cell,比较两者的输出状态
# staste_is_tuple 表示将输出状态和输出结果合并,变成(状态,输出值)
cell = tf.contrib.rnn.BasicLSTMCell(num_units=3,state_is_tuple=True)
gru = tf.contrib.rnn.GRUCell(num_units=3)

# 如果没有 initial_state,必须指定 dtype
outputs,last_states = tf.nn.dynamic_rnn(cell,x,sequence_length=seq_lengths,dtype=tf.float32)
gru_outputs,gru_last_states = tf.nn.dynamic_rnn(gru,x,sequence_length=seq_lengths,dtype=tf.float32)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
output,cell_state,gru_output,gru_state = sess.run([outputs,last_states,gru_outputs,gru_last_states])
print("full_seq:")
print(output[0])
print("short_seq:")
print(output[1])
print("lstm_cell_state:")
print(len(cell_state))
print(cell_state[1])
print("gru_short_cell_state:")
print(gru_output[1])
print("gru_cell_state:")
print(len(gru_state))
print(gru_state[1])

full_seq:
[[-0.08356715 -0.09119065  0.014137  ]
 [-0.0247704   0.18466851 -0.10629261]
 [-0.17865723 -0.18755475 -0.10416328]
 [ 0.15664953 -0.05130756  0.10724574]]
short_seq:
[[-0.02775398  0.13164161  0.0717001 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
lstm_cell_state:
2
[[ 0.15664953 -0.05130756  0.10724574]
 [-0.02775398  0.13164161  0.0717001 ]]
gru_short_cell_state:
[[-0.0616577   0.01478057 -0.07920398]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
gru_cell_state:
2
[-0.0616577   0.01478057 -0.07920398]


### 6 构建单层静态/动态 LSTM 网络对 MNIST 数据集分类

In [8]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/mnist/",one_hot = True)
tf.reset_default_graph()

n_input = 28
n_steps = 28
n_hidden = 128
n_classes = 10

x = tf.placeholder(dtype=tf.float32,shape=[None,n_steps*n_input])
y = tf.placeholder(dtype=tf.float32,shape=[None,n_classes])

# 如果使用 static_rnn 就需要将输入转换成 [timesteps,batch_size,cell_data]
# 如果使用 dynamic_rnn 就不需要转换
# tf.unstack 表示将输入x 按照 axis=1 拆分成 n_steps 个元素
x_1 = tf.reshape(x,[-1,n_steps,n_input])
x_2 = tf.unstack(x_1,n_steps,axis=1)
# 由于 static_rnn 的输入数据必须是一个 list 所以不能直接使用 tf.transpose 转换
# x_2 = tf.transpose(x_1,[1,0,2])

lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_hidden)
# gru_cell = tf.contrib.rnn.GRUCell(num_units=n_hidden)
outputs,states = tf.nn.static_rnn(cell=lstm_cell,inputs=x_2,dtype=tf.float32)
# 静态 RNN 的输出,形状为[timesteps,batch_size,...],所以直接取最后一步的结果就可以了
# 注意动态的输入数据格式为[batch_size,timesteps,cell_data]
# outputs,states = tf.nn.dynamic_rnn(cell=lstm_cell,inputs=x_1,dtype=tf.float32)
# 动态 RNN 的输出,形状为[batch_size,timesteps,...],需要转换结果,变成[timesteps,batch_size,...]
# 一般可以使用 tf.transpose 转换,可以直接换位置,或者也可以使用 tf.unstack
# outputs = tf.transpose(outputs,[1,0,2])
pred = tf.contrib.layers.fully_connected(outputs[-1],n_classes,activation_fn=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

train_epochs = 20
batch_size = 100
display_step = 1
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(train_epochs):
        avg_cost = 0.0
        total_batch = int(mnist.train.num_examples/batch_size)
        for j in range(total_batch):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            _,loss = sess.run([optimizer,cost],feed_dict={x:batch_x,y:batch_y})
            avg_cost = avg_cost+loss/total_batch
        if (i+1)%display_step==0:
            # 如果两者最大值索引相等,那么就会该位置就会变成True,否则变成False
            correct_prediction = tf.equal(tf.argmax(pred,axis=1),tf.argmax(y,axis=1))
            acc = tf.reduce_mean(tf.cast(correct_prediction,dtype=tf.float32))
            accuracy = sess.run(acc,feed_dict={x:mnist.test.images,y:mnist.test.labels})
            print('epochs:',i+1,'loss:',avg_cost,'acc:',accuracy)

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz
('epochs:', 1, 'loss:', 0.32108588239008684, 'acc:', 0.9754)
('epochs:', 2, 'loss:', 0.08950058505616408, 'acc:', 0.9813)
('epochs:', 3, 'loss:', 0.06395129573235121, 'acc:', 0.98)
('epochs:', 4, 'loss:', 0.05754251313522797, 'acc:', 0.9816)
('epochs:', 5, 'loss:', 0.04936414777872744, 'acc:', 0.9797)
('epochs:', 6, 'loss:', 0.04605702987967314, 'acc:', 0.9829)
('epochs:', 7, 'loss:', 0.04210281130963601, 'acc:', 0.978)
('epochs:', 8, 'loss:', 0.041488120896022086, 'acc:', 0.9858)
('epochs:', 9, 'loss:', 0.03870155098096634, 'acc:', 0.9847)
('epochs:', 10, 'loss:', 0.04109361199640925, 'acc:', 0.9789)
('epochs:', 11, 'loss:', 0.0406065006763674, 'acc:', 0.9834)
('epochs:', 12, 'loss:', 0.03408122316129846, 'acc:', 0.9843)
('epochs:', 13, 'loss:', 0.038966907959812426, 'acc:', 0.9854)
('epochs:',

### 7 构建多层静态/动态 LSTM 网络对 MNIST 数据集分类

多层 RNN 在创建过程中,需要使用到前面介绍的 MultiRNNCell ,这个类实例化需要通过单层的 cell 对象输入.
与前面的例子类似,先创建单层的 cell,然后再创建 MultiRNNCell 对象,在创建好 MultiRNNCell 后,可以通过静态或动态的 RNN 网络组合.

In [9]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/mnist/",one_hot = True)
tf.reset_default_graph()

n_input = 28
n_steps = 28
n_hidden = 128
n_classes = 10

x = tf.placeholder(dtype=tf.float32,shape=[None,n_steps*n_input])
y = tf.placeholder(dtype=tf.float32,shape=[None,n_classes])

# 如果使用 static_rnn 就需要将输入转换成 [timesteps,batch_size,cell_data]
# 如果使用 dynamic_rnn 就不需要转换
# tf.unstack 表示将输入x 按照 axis=1 拆分成 n_steps 个元素
x_1 = tf.reshape(x,[-1,n_steps,n_input])
x_2 = tf.unstack(x_1,n_steps,axis=1)
# 由于 static_rnn 的输入数据必须是一个 list 所以不能直接使用 tf.transpose 转换
# x_2 = tf.transpose(x_1,[1,0,2])

# 不能直接使用 [cell]*3 会导致作用域冲突
stack_rnn = []
for i in range(3):
    stack_rnn.append(tf.contrib.rnn.LSTMCell(num_units=n_hidden))
# 使用 MultiRNNCell 构建多层
mcell = tf.contrib.rnn.MultiRNNCell(stack_rnn)
outputs,states = tf.nn.static_rnn(cell=mcell,inputs=x_2,dtype=tf.float32)
# 静态 RNN 的输出,形状为[timesteps,batch_size,...],所以直接取最后一步的结果就可以了
# 注意动态的输入数据格式为[batch_size,timesteps,cell_data]
# outputs,states = tf.nn.dynamic_rnn(cell=lstm_cell,inputs=x_1,dtype=tf.float32)
# 动态 RNN 的输出,形状为[batch_size,timesteps,...],需要转换结果,变成[timesteps,batch_size,...]
# 一般可以使用 tf.transpose 转换,可以直接换位置,或者也可以使用 tf.unstack
# outputs = tf.transpose(outputs,[1,0,2])
pred = tf.contrib.layers.fully_connected(outputs[-1],n_classes,activation_fn=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

train_epochs = 20
batch_size = 100
display_step = 1
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(train_epochs):
        avg_cost = 0.0
        total_batch = int(mnist.train.num_examples/batch_size)
        for j in range(total_batch):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            _,loss = sess.run([optimizer,cost],feed_dict={x:batch_x,y:batch_y})
            avg_cost = avg_cost+loss/total_batch
        if (i+1)%display_step==0:
            # 如果两者最大值索引相等,那么就会该位置就会变成True,否则变成False
            correct_prediction = tf.equal(tf.argmax(pred,axis=1),tf.argmax(y,axis=1))
            acc = tf.reduce_mean(tf.cast(correct_prediction,dtype=tf.float32))
            accuracy = sess.run(acc,feed_dict={x:mnist.test.images,y:mnist.test.labels})
            print('epochs:',i+1,'loss:',avg_cost,'acc:',accuracy)

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz
('epochs:', 1, 'loss:', 0.382035752863369, 'acc:', 0.96)
('epochs:', 2, 'loss:', 0.12375072764910074, 'acc:', 0.9723)
('epochs:', 3, 'loss:', 0.10376593740741642, 'acc:', 0.969)
('epochs:', 4, 'loss:', 0.0962252793715082, 'acc:', 0.9777)
('epochs:', 5, 'loss:', 0.09086501511745156, 'acc:', 0.9783)
('epochs:', 6, 'loss:', 0.09430076590569858, 'acc:', 0.9725)
('epochs:', 7, 'loss:', 0.09274781296961002, 'acc:', 0.9713)
('epochs:', 8, 'loss:', 0.13535508378578184, 'acc:', 0.9561)
('epochs:', 9, 'loss:', 0.13893080826002097, 'acc:', 0.9292)
('epochs:', 10, 'loss:', 0.9730930625579577, 'acc:', 0.7425)
('epochs:', 11, 'loss:', 0.4223040672865779, 'acc:', 0.9202)
('epochs:', 12, 'loss:', 0.23397439456121472, 'acc:', 0.9319)
('epochs:', 13, 'loss:', 0.1646895441683856, 'acc:', 0.9512)
('epochs:', 14, 'l

### 8 构建单层静态/动态双向 RNN 网络对 MNIST 数据集分类

先建立两个 cell ,每个 cell 包含 128 个隐含单元,然后使用 $tf.nn.bidirection\_dynamic\_rnn$ 函数将 $x$ 放进去生成节点 outputs, 由于 bidirectional_dynamic_rnn 的输出结果与状态是分离的, 所以需要手动将结果合并起来并进行转置, 然后通过全连接层生成 pred,再使用 softmax 进行分类.

In [10]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/mnist/",one_hot = True)
tf.reset_default_graph()

n_input = 28
n_steps = 28
n_hidden = 128
n_classes = 10

x = tf.placeholder(dtype=tf.float32,shape=[None,n_steps*n_input])
y = tf.placeholder(dtype=tf.float32,shape=[None,n_classes])
x_1 = tf.reshape(x,[-1,n_steps,n_input])
# 前向cell
lstm_fw_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden)
# 反向cell
lstm_bw_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden)
# 构建双向 RNN 的函数
outputs,states = tf.nn.bidirectional_dynamic_rnn(cell_fw=lstm_fw_cell,cell_bw=lstm_bw_cell,
                                                 inputs=x_1,dtype=tf.float32)
# 将输出结果的那一个维度合并起来
outputs = tf.concat(outputs,axis=2)
outputs = tf.transpose(outputs,[1,0,2])
pred = tf.contrib.layers.fully_connected(outputs[-1],n_classes,activation_fn=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

train_epochs = 20
batch_size = 100
display_step = 1
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(train_epochs):
        avg_cost = 0.0
        total_batch = int(mnist.train.num_examples/batch_size)
        for j in range(total_batch):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            _,loss = sess.run([optimizer,cost],feed_dict={x:batch_x,y:batch_y})
            avg_cost = avg_cost+loss/total_batch
        if (i+1)%display_step==0:
            # 如果两者最大值索引相等,那么就会该位置就会变成True,否则变成False
            correct_prediction = tf.equal(tf.argmax(pred,axis=1),tf.argmax(y,axis=1))
            acc = tf.reduce_mean(tf.cast(correct_prediction,dtype=tf.float32))
            accuracy = sess.run(acc,feed_dict={x:mnist.test.images,y:mnist.test.labels})
            print('epochs:',i+1,'loss:',avg_cost,'acc:',accuracy)

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz
('epochs:', 1, 'loss:', 0.32167011274363516, 'acc:', 0.9692)
('epochs:', 2, 'loss:', 0.09290599693036218, 'acc:', 0.9767)
('epochs:', 3, 'loss:', 0.07222473755648195, 'acc:', 0.9804)
('epochs:', 4, 'loss:', 0.06027137368028478, 'acc:', 0.9783)
('epochs:', 5, 'loss:', 0.04935369869757609, 'acc:', 0.9819)
('epochs:', 6, 'loss:', 0.046610671895429105, 'acc:', 0.9838)
('epochs:', 7, 'loss:', 0.0426436970689842, 'acc:', 0.9825)
('epochs:', 8, 'loss:', 0.03908720361746166, 'acc:', 0.9853)
('epochs:', 9, 'loss:', 0.04210371764768338, 'acc:', 0.9829)
('epochs:', 10, 'loss:', 0.041152892129347536, 'acc:', 0.9815)
('epochs:', 11, 'loss:', 0.0383973893146454, 'acc:', 0.9847)
('epochs:', 12, 'loss:', 0.04132545971269298, 'acc:', 0.9843)
('epochs:', 13, 'loss:', 0.03511075249407445, 'acc:', 0.9833)
('epochs: